# Star Wars Chatbot in Jupyter Notebook
This notebook implements a simple chatbot using intents from a JSON file.

In [1]:
# Import necessary libraries
import json
import random
import nltk
import numpy as np
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import pickle
import os

nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\a\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\a\AppData\Roaming\nltk_data...


True

In [2]:
# Load dataset
with open('starwarsintents.json', 'r') as file:
    intents = json.load(file)

print(intents)  # Display structure of the intents JSON file


{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day', "What's up", 'Yo!', 'Howdy', 'Nice to meet you.'], 'responses': ['Hey', 'Hello, thanks for visiting.', 'Hi there, what can I do for you?', 'Hi there, how can I help?', 'Hello, there.', 'Hello Dear', 'Ooooo Hello, looking for someone or something?', 'Yes, I am here.', 'Listening carefully.', 'Ok, I am with you.']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later.', 'Goodbye', 'Have a great day.', 'See you next time.', 'It was my pleassure.', 'Take care.', 'See ya!', 'Catch you later.', 'Ciao.'], 'responses': ['See you later, thanks for visiting.', 'May the force be with you!', 'See next time.', 'Was my pleassuare to meet you.', 'Hope will cath up sortly.', 'Have a nice day.', 'Bye! Come back again soon.', 'So, till next time.', 'If you need anything just text me anytime. Bye.', 'Well, hope see you soon!']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's

In [8]:
nltk.download('punkt')
nltk.download('all')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\a\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\a\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\a\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\a\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\a\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tag

True

In [9]:
# Data preprocessing
lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

print(f"Classes: {classes}")
print(f"Words: {words}")


Classes: ['Menu', 'about me', 'alive', 'bounti hounter', 'creator', 'funny', 'goodbye', 'greeting', 'hepl', 'jedi', 'mission', 'myself', 'sith', 'stories', 'tasks', 'thanks']
Words: ["'s", '10', 'a', 'ability', 'able', 'about', 'alive', 'am', 'amazing', 'any', 'anyone', 'are', 'aslan', 'assist', 'assistance', 'bar', 'be', 'best', 'bit', 'bounti', 'breathe', 'bye', 'can', 'care', 'catch', 'check', 'ciao', 'created', 'creator', 'daddy', 'day', 'detail', 'do', 'drink', 'else', 'father', 'feature', 'for', 'funny', 'galaxi', 'galaxy', 'good', 'goodbye', 'great', 'have', 'hello', 'help', 'helpful', 'hey', 'hi', 'hope', 'hounter', 'how', 'howdy', 'i', 'identify', 'in', 'is', 'it', 'item', 'jedi', 'joke', 'kind', 'know', 'later', 'let', 'looking', 'lot', 'me', 'meet', 'menu', 'mision', 'mission', 'mr.', 'my', 'myself', 'need', 'next', 'nice', 'now', 'of', 'on', 'one', 'partner', 'person', 'please', 'pleassure', 'profile', 'right', 'run', 'see', 'selection', 'serve', 'sing', 'sith', 'so', 'some

In [10]:
# Create training data
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

print("Training data prepared.")


Training data prepared.


In [11]:
# Build neural network model
model = Sequential([
    Dense(128, input_shape=(len(train_x[0]),), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(train_y[0]), activation='softmax')
])

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save("chatbot_model.h5")
print("Model created and saved.")


C:\Users\a\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200


C:\Users\a\AppData\Roaming\Python\Python312\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0823 - loss: 2.7460  
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1581 - loss: 2.6449
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1693 - loss: 2.5986    
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2283 - loss: 2.4536
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3295 - loss: 2.3244 
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3992 - loss: 2.2372 
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4265 - loss: 2.0983
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4064 - loss: 1.8155   
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3708 - loss: 1.9366
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4214 - loss: 1.6469 
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6047 - loss: 1.3778
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy:

Model created and saved.


In [12]:
# Function to predict responses
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, word in enumerate(words):
            if word == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence, model):
    bow = bag_of_words(sentence, words)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    for i in intents_json['intents']:
        if i['tag'] == tag:
            return random.choice(i['responses'])
    return "I'm not sure how to respond to that."

print("Chatbot is ready!")


Chatbot is ready!


In [13]:
# Chatbot interaction
print("Start chatting with the bot (type 'quit' to stop)!")
while True:
    message = input("You: ")
    if message.lower() == "quit":
        break
    ints = predict_class(message, model)
    res = get_response(ints, intents)
    print(f"Bot: {res}")


Start chatting with the bot (type 'quit' to stop)!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
Bot: Ooooo Hello, looking for someone or something?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Bot: Ok, I am with you.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Bot: Hi there, how can I help?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Bot: I was created by Mr. ASLAN.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
Bot: I was created by Mr. ASLAN.
